<a href="https://colab.research.google.com/github/ale-camer/Finance/blob/master/Volatility_Surface_(not_calculated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing package if required to download financial data

import pandas as pd

packages = !pip list -v
packagesList = []
for i in range(len(packages)):
  packagesList.append(packages[i].split())
packagesList = pd.DataFrame(packagesList)
packagesList.columns = packagesList.iloc[0,:]
packagesList = packagesList.iloc[2:,:]

if 'yfinance' in packagesList.iloc[:,0].values:
  print("yfinance package was already installed.")
  pass
else:
  !pip install -q yfinance
  print("yfinance package was installed")

     |████████████████████████████████| 62 kB 291 kB/s 
     |████████████████████████████████| 6.4 MB 23.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
yfinance package was installed


In [ ]:
def expVolSurf():

  """ Provide 3D plot of the volatilty surface of calls and puts of any stock listed in the New York Stock Exchange (NYSE).
    The whole data is download and calculated by Yahoo Finance.
    Only is considered options that were traded in the last business day. """

  try:

    import pandas as pd
    import plotly.graph_objects as go
    import warnings
    warnings.filterwarnings('ignore')
    import yfinance as yf
    from sklearn.impute import KNNImputer
    from tqdm import tqdm
    import sys
    from plotly.offline import iplot

    imputer = KNNImputer(n_neighbors=5)
    ticker = input("Please, insert the Ticker: ")
    aapl = yf.Ticker(ticker)
    dates = aapl.options

    print("Downloading data")
    calls, puts = pd.DataFrame(), pd.DataFrame()
    for date in tqdm(range(len(dates))):
        try:
            calls = calls.append(aapl.option_chain(dates[date]).calls,ignore_index=True)
            puts = puts.append(aapl.option_chain(dates[date]).puts,ignore_index=True)
        except:
            continue  

    print("Processing data")
    lastTradeDate = pd.Series(pd.DataFrame(calls['lastTradeDate'])['lastTradeDate'].apply(lambda a: a.strftime('%Y-%m-%d')).unique()).sort_values(ascending=False)[0]
    calls, puts = calls[calls['lastTradeDate'].apply(lambda a: a.strftime('%Y-%m-%d')) == lastTradeDate], puts[puts['lastTradeDate'].apply(lambda a: a.strftime('%Y-%m-%d')) == lastTradeDate]
    calls['lastTradeDate'], puts['lastTradeDate'] = calls['lastTradeDate'].apply(lambda a: a.strftime('%Y-%m-%d')), puts['lastTradeDate'].apply(lambda a: a.strftime('%Y-%m-%d'))
    calls['expirationDate'], puts['expirationDate'] = '20' + calls['contractSymbol'].apply(lambda a: a[-15:][:2] + '-' + a[-13:][:2] + '-' + a[-11:][:2]), \
    '20' + puts['contractSymbol'].apply(lambda a: a[-15:][:2] + '-' + a[-13:][:2] + '-' + a[-11:][:2])
    strikes = calls.strike.value_counts()[calls.strike.value_counts() >= calls.strike.value_counts().values[:10].min()].index
    calls, puts = calls[calls.strike.isin(strikes)], puts[puts.strike.isin(strikes)]
    calls, puts = pd.pivot_table(data = calls, values = 'impliedVolatility', columns = 'strike', index = 'expirationDate'), \
    pd.pivot_table(data = puts, values = 'impliedVolatility', columns = 'strike', index = 'expirationDate')

    na_prop = (calls.isna().sum().sum() / (calls.shape[0] * calls.shape[1]) + puts.isna().sum().sum() / (puts.shape[0] * puts.shape[1])) / 2
    if na_prop >= 0.4:
      sys.exit()
    else:
      pass

    calls1, puts1 = pd.DataFrame(imputer.fit_transform(calls)), pd.DataFrame(imputer.fit_transform(puts))
    calls1.index, calls1.columns, puts1.index, puts1.columns = calls.index, calls.columns, puts.index, puts.columns

    print("Printing images")
    xc, yc, zc = calls1.columns, calls1.index.values, calls1.values
    xp, yp, zp = puts1.columns, puts1.index.values, puts1.values

    layout = go.Layout(xaxis = go.layout.XAxis(
        title = go.layout.xaxis.Title(
            text = 'Date')),
        yaxis = go.layout.YAxis(
            title = go.layout.yaxis.Title(
                text = 'Strike')))

    fig = go.Figure(data = [go.Surface(z = zc, y = yc, x = xc)], layout = layout)
    fig.update_layout(title = f'{ticker.upper()} Call Implied Volatility Surface',
                      scene = dict(
                        xaxis_title = 'Strike',
                        yaxis_title = 'Date',
                        zaxis_title = 'Implied Volatility'),
                      autosize = False,
                      width = 800, height = 800,
                      margin = dict(l = 65, r = 50, b = 65, t = 90))
    iplot(fig)

    fig = go.Figure(data= [go.Surface(z = zp, y = yp, x = xp)], layout = layout)
    fig.update_layout(title = f'{ticker.upper()} Put Implied Volatility Surface',
                      scene = dict(
                        xaxis_title = 'Strike',
                        yaxis_title = 'Date',
                        zaxis_title = 'Implied Volatility'),
                      autosize = False,
                      width = 800, height = 800,
                      margin = dict(l = 65, r = 50, b = 65, t = 90))
    iplot(fig)

  except:

    print("\n Not enough data available. Please, run the program again and choose another ticker.")

In [7]:
expVolSurf()

Please, insert the Ticker: aapl


100%|██████████| 18/18 [00:04<00:00,  4.28it/s]


Processing data
Printing images
